In [1]:
from redis import Redis
REDIS = Redis('this_redis')

# Redis

One interesting use case is to have Redis track the evolution of some long running iterative process.

In [2]:
REDIS.get('count')

b'11'

In [3]:
REDIS.get('count')

b'12'

In [4]:
REDIS.get('count')

b'14'

In [5]:
REDIS.get('count')

b'30'

In [6]:
REDIS.get('count')

b'30'

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [7]:
model_params_from_redis = REDIS.get('model_params')

In [8]:
type(model_params_from_redis)

bytes

In [9]:
type(model_params_from_redis.decode())

str

In [10]:
import json

In [11]:
json.loads(model_params_from_redis.decode())

{'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [12]:
import numpy as np

In [13]:
A_bytestring = REDIS.get('encoded_A')

In [19]:
A_raveled = np.fromstring(A_bytestring, dtype=int)

In [20]:
n = int(REDIS.get('A_n').decode())
m = int(REDIS.get('A_m').decode())

In [21]:
A = A_raveled.reshape(n, m)

In [22]:
A

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [23]:
def matrix_from_redis(redis_client, key, dtype=int):
    matrix_bytestring = redis_client.get(key)
    matrix_raveled = np.fromstring(matrix_bytestring, dtype=dtype)
    n = int(redis_client.get(key+'_n').decode())
    m = int(redis_client.get(key+'_m').decode())
    matrix = matrix_raveled.reshape(n, m)
    return matrix

In [24]:
matrix_from_redis(REDIS, 'new_A')

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

This could also be done via the `pickle` library.

In [25]:
import pickle

B_pkl = REDIS.get('B')
pickle.loads(B_pkl)

array([[7, 6, 8],
       [5, 5, 1],
       [4, 2, 4]])